# Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [22]:
import pandas as pd

In [23]:
data_movies = pd.read_csv('/Users/viktoria/Documents/Netology/ht_repository/ml-latest-small/movies.csv')
data_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [77]:
data_ratings = pd.read_csv('/Users/viktoria/Documents/Netology/ht_repository/ml-latest-small/ratings.csv').groupby(['movieId'])['rating'].mean().reset_index()
data_ratings.head()

,movieId,rating
0,1,3.872470
1,2,3.401869
2,3,3.161017
3,4,2.384615
4,5,3.267857


In [78]:
data_final = data_movies.merge(data_ratings, on="movieId", how = 'inner')
data_final.head()

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615
4,5,Father of the Bride Part II (1995),Comedy,3.267857


In [79]:
def class_movies(rating):
    if rating <= 2.0:
        return 'низкий рейтинг'
    if rating <= 4.0:
        return 'средний рейтинг'
    if rating <= 5.0:
        return 'высокий рейтинг'

In [83]:
data_final['class'] = data_final['rating'].apply(class_movies)
data_final.head()

,movieId,title,genres,rating,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,3.267857,средний рейтинг


# Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Результат классификации запишите в отдельный столбец region.

In [58]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
csv_url = '/Users/viktoria/Documents/Netology/ht_repository/ml-latest-small/keywords.csv'

def merge_geo_data(keys_csv, geodata):
    
    def find_region_by_city(city):
        for (reg, cities) in geodata.items():
            for c in cities:
                if c in city:
                    return reg
        return 'undefined'
    
    keyset = pd.read_csv(keys_csv)
    keyset['region'] = keyset['keyword'].apply(find_region_by_city)
    return keyset

r = merge_geo_data(csv_url, geo_data)
r[(r.region != 'undefined')].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


# Задание 3

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

In [92]:
years = range(1950, 2030)

def production_year(s):
    for y in years:
        if str(y) in s:
            return y
    return 1900

data_final['year'] = data_final['title'].apply(production_year)
data_final.groupby(['year'])['rating'].mean().reset_index().sort_values(by=['rating'], ascending=False).head()

,year,rating
2,1951,3.906653
3,1952,3.802318
13,1962,3.772962
4,1953,3.738878
8,1957,3.693322
